In [ ]:
#This notebook trains all the five models on the dataset, shows their accuracy and also shows the accuracy after ensembling

In [1]:
import tensorflow as tf
import sklearn.model_selection as skms
import sklearn.preprocessing as skp
from tensorflow import keras
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import pandas as pd
import numpy as np
import random
seed = 12
np.random.seed(seed)
from sklearn import model_selection as skms

In [6]:
df = pd.read_csv('features_3_sec.csv')
df.head()

,filename,length,chroma_stft_mean,chroma_stft_var,rms_mean,rms_var,spectral_centroid_mean,spectral_centroid_var,spectral_bandwidth_mean,spectral_bandwidth_var,...,mfcc16_var,mfcc17_mean,mfcc17_var,mfcc18_mean,mfcc18_var,mfcc19_mean,mfcc19_var,mfcc20_mean,mfcc20_var,label
0,blues.00000.0.wav,66149,0.335406,0.091048,0.130405,0.003521,1773.065032,167541.630869,1972.744388,117335.771563,...,39.687145,-3.241280,36.488243,0.722209,38.099152,-5.050335,33.618073,-0.243027,43.771767,blues
1,blues.00000.1.wav,66149,0.343065,0.086147,0.112699,0.001450,1816.693777,90525.690866,2010.051501,65671.875673,...,64.748276,-6.055294,40.677654,0.159015,51.264091,-2.837699,97.030830,5.784063,59.943081,blues
2,blues.00000.2.wav,66149,0.346815,0.092243,0.132003,0.004620,1788.539719,111407.437613,2084.565132,75124.921716,...,67.336563,-1.768610,28.348579,2.378768,45.717648,-1.938424,53.050835,2.517375,33.105122,blues
3,blues.00000.3.wav,66149,0.363639,0.086856,0.132565,0.002448,1655.289045,111952.284517,1960.039988,82913.639269,...,47.739452,-3.841155,28.337118,1.218588,34.770935,-3.580352,50.836224,3.630866,32.023678,blues
4,blues.00000.4.wav,66149,0.335579,0.088129,0.143289,0.001701,1630.656199,79667.267654,1948.503884,60204.020268,...,30.336359,0.664582,45.880913,1.689446,51.363583,-3.392489,26.738789,0.536961,29.146694,blues


In [8]:
label_index = dict()
index_label = dict()
for i, x in enumerate(df.label.unique()):
    label_index[x] = i
    index_label[i] = x
print(label_index)
print(index_label)

{'blues': 0, 'classical': 1, 'country': 2, 'disco': 3, 'hiphop': 4, 'jazz': 5, 'metal': 6, 'pop': 7, 'reggae': 8, 'rock': 9}
{0: 'blues', 1: 'classical', 2: 'country', 3: 'disco', 4: 'hiphop', 5: 'jazz', 6: 'metal', 7: 'pop', 8: 'reggae', 9: 'rock'}


In [9]:
df.label = [label_index[l] for l in df.label]

In [11]:
df_shuffle = df.sample(frac=1, random_state=11).reset_index(drop=True)

In [12]:
df_X=df_shuffle.iloc[:,2:-1]
df_y=df_shuffle.iloc[:,-1]

In [13]:
X_train, X_test, y_train, y_test = skms.train_test_split(df_X, df_y, train_size=0.9, random_state=seed, stratify=df_y)

In [14]:
from sklearn import preprocessing as skp
scaler = skp.StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test =scaler.transform(X_test)

In [18]:
model=keras.Sequential([
    keras.layers.Dense(256,input_shape=(X_train.shape[1],),activation='relu'),
    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dense(10, activation='softmax')
    
    
])

model.compile(
optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

model.fit(X_train,y_train,epochs=70,validation_data=(X_test, y_test))

Epoch 1/70
281/281 [==============================] - 1s 2ms/step - loss: 1.0688 - accuracy: 0.6317 - val_loss: 0.7377 - val_accuracy: 0.7457
Epoch 2/70
281/281 [==============================] - 0s 2ms/step - loss: 0.6344 - accuracy: 0.7836 - val_loss: 0.6440 - val_accuracy: 0.7768
Epoch 3/70
281/281 [==============================] - 0s 2ms/step - loss: 0.4860 - accuracy: 0.8332 - val_loss: 0.5483 - val_accuracy: 0.8158
Epoch 4/70
281/281 [==============================] - 0s 2ms/step - loss: 0.3729 - accuracy: 0.8740 - val_loss: 0.5283 - val_accuracy: 0.8118
Epoch 5/70
281/281 [==============================] - 0s 2ms/step - loss: 0.2969 - accuracy: 0.9008 - val_loss: 0.4632 - val_accuracy: 0.8448
Epoch 6/70
281/281 [==============================] - 0s 2ms/step - loss: 0.2395 - accuracy: 0.9213 - val_loss: 0.4020 - val_accuracy: 0.8679
Epoch 7/70
281/281 [==============================] - 0s 2ms/step - loss: 0.1835 - accuracy: 0.9403 - val_loss: 0.4622 - val_accuracy: 0.8649
Epoch 

In [19]:
model_2 = keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(10, activation='softmax'),
])

model_2.compile(
optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model_2.fit(X_train,y_train,epochs=100,validation_data=(X_test, y_test))



Epoch 1/100
281/281 [==============================] - 2s 4ms/step - loss: 1.3456 - accuracy: 0.5224 - val_loss: 0.8312 - val_accuracy: 0.7207
Epoch 2/100
281/281 [==============================] - 1s 3ms/step - loss: 0.9081 - accuracy: 0.6908 - val_loss: 0.6723 - val_accuracy: 0.7838
Epoch 3/100
281/281 [==============================] - 1s 3ms/step - loss: 0.7296 - accuracy: 0.7541 - val_loss: 0.5595 - val_accuracy: 0.8038
Epoch 4/100
281/281 [==============================] - 1s 3ms/step - loss: 0.6052 - accuracy: 0.8009 - val_loss: 0.5322 - val_accuracy: 0.8208
Epoch 5/100
281/281 [==============================] - 1s 3ms/step - loss: 0.5341 - accuracy: 0.8204 - val_loss: 0.4914 - val_accuracy: 0.8318
Epoch 6/100
281/281 [==============================] - 1s 3ms/step - loss: 0.4647 - accuracy: 0.8482 - val_loss: 0.4629 - val_accuracy: 0.8378
Epoch 7/100
281/281 [==============================] - 1s 3ms/step - loss: 0.4115 - accuracy: 0.8648 - val_loss: 0.3968 - val_accuracy: 0.8639

Epoch 58/100
281/281 [==============================] - 1s 4ms/step - loss: 0.0642 - accuracy: 0.9795 - val_loss: 0.3005 - val_accuracy: 0.9239
Epoch 59/100
281/281 [==============================] - 2s 6ms/step - loss: 0.0642 - accuracy: 0.9819 - val_loss: 0.3168 - val_accuracy: 0.9209
Epoch 60/100
281/281 [==============================] - 1s 4ms/step - loss: 0.0725 - accuracy: 0.9764 - val_loss: 0.2626 - val_accuracy: 0.9349
Epoch 61/100
281/281 [==============================] - 1s 3ms/step - loss: 0.0609 - accuracy: 0.9823 - val_loss: 0.3466 - val_accuracy: 0.9239
Epoch 62/100
281/281 [==============================] - 1s 5ms/step - loss: 0.0530 - accuracy: 0.9831 - val_loss: 0.3401 - val_accuracy: 0.9269
Epoch 63/100
281/281 [==============================] - 2s 6ms/step - loss: 0.0617 - accuracy: 0.9801 - val_loss: 0.3642 - val_accuracy: 0.9249
Epoch 64/100
281/281 [==============================] - 1s 3ms/step - loss: 0.0563 - accuracy: 0.9828 - val_loss: 0.3432 - val_accuracy:

In [20]:
model_3 = keras.models.Sequential([
    keras.layers.Dense(512, activation='relu', input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.2),
    
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(128, activation='relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.2),

    keras.layers.Dense(10, activation='softmax'),
])

model_3.compile(
optimizer='sgd',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)
model_3.fit(X_train,y_train,epochs=700,validation_data=(X_test, y_test))

Epoch 1/700
281/281 [==============================] - 1s 3ms/step - loss: 2.0981 - accuracy: 0.2409 - val_loss: 1.7175 - val_accuracy: 0.3814
Epoch 2/700
281/281 [==============================] - 1s 3ms/step - loss: 1.6418 - accuracy: 0.4041 - val_loss: 1.3532 - val_accuracy: 0.5315
Epoch 3/700
281/281 [==============================] - 1s 3ms/step - loss: 1.4416 - accuracy: 0.4744 - val_loss: 1.1802 - val_accuracy: 0.5936
Epoch 4/700
281/281 [==============================] - 1s 3ms/step - loss: 1.3139 - accuracy: 0.5257 - val_loss: 1.0671 - val_accuracy: 0.6577
Epoch 5/700
281/281 [==============================] - 1s 3ms/step - loss: 1.2122 - accuracy: 0.5741 - val_loss: 0.9889 - val_accuracy: 0.6697
Epoch 6/700
281/281 [==============================] - 1s 3ms/step - loss: 1.1340 - accuracy: 0.6035 - val_loss: 0.9096 - val_accuracy: 0.7117
Epoch 7/700
281/281 [==============================] - 1s 3ms/step - loss: 1.0657 - accuracy: 0.6260 - val_loss: 0.8568 - val_accuracy: 0.7247

Epoch 58/700
281/281 [==============================] - 1s 3ms/step - loss: 0.2442 - accuracy: 0.9198 - val_loss: 0.3113 - val_accuracy: 0.9009
Epoch 59/700
281/281 [==============================] - 1s 3ms/step - loss: 0.2451 - accuracy: 0.9215 - val_loss: 0.3145 - val_accuracy: 0.9009
Epoch 60/700
281/281 [==============================] - 1s 5ms/step - loss: 0.2321 - accuracy: 0.9240 - val_loss: 0.3100 - val_accuracy: 0.9069
Epoch 61/700
281/281 [==============================] - 1s 3ms/step - loss: 0.2227 - accuracy: 0.9245 - val_loss: 0.3142 - val_accuracy: 0.9029
Epoch 62/700
281/281 [==============================] - 1s 3ms/step - loss: 0.2127 - accuracy: 0.9298 - val_loss: 0.2986 - val_accuracy: 0.9109
Epoch 63/700
281/281 [==============================] - 1s 5ms/step - loss: 0.2079 - accuracy: 0.9300 - val_loss: 0.3083 - val_accuracy: 0.9129
Epoch 64/700
281/281 [==============================] - 1s 3ms/step - loss: 0.2135 - accuracy: 0.9306 - val_loss: 0.3235 - val_accuracy:

Epoch 115/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0908 - accuracy: 0.9703 - val_loss: 0.2622 - val_accuracy: 0.9309
Epoch 116/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0935 - accuracy: 0.9726 - val_loss: 0.2614 - val_accuracy: 0.9299
Epoch 117/700
281/281 [==============================] - 1s 4ms/step - loss: 0.0805 - accuracy: 0.9739 - val_loss: 0.2760 - val_accuracy: 0.9269
Epoch 118/700
281/281 [==============================] - 1s 4ms/step - loss: 0.0833 - accuracy: 0.9734 - val_loss: 0.2720 - val_accuracy: 0.9259
Epoch 119/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0785 - accuracy: 0.9743 - val_loss: 0.2778 - val_accuracy: 0.9229
Epoch 120/700
281/281 [==============================] - 1s 5ms/step - loss: 0.0803 - accuracy: 0.9725 - val_loss: 0.2529 - val_accuracy: 0.9369
Epoch 121/700
281/281 [==============================] - 1s 4ms/step - loss: 0.0799 - accuracy: 0.9744 - val_loss: 0.2700 - val_ac

281/281 [==============================] - 1s 4ms/step - loss: 0.0349 - accuracy: 0.9900 - val_loss: 0.2483 - val_accuracy: 0.9439
Epoch 228/700
281/281 [==============================] - 2s 5ms/step - loss: 0.0328 - accuracy: 0.9903 - val_loss: 0.2568 - val_accuracy: 0.9429
Epoch 229/700
281/281 [==============================] - 1s 4ms/step - loss: 0.0292 - accuracy: 0.9909 - val_loss: 0.2624 - val_accuracy: 0.9399
Epoch 230/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0297 - accuracy: 0.9912 - val_loss: 0.2687 - val_accuracy: 0.9419
Epoch 231/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0303 - accuracy: 0.9901 - val_loss: 0.2580 - val_accuracy: 0.9429
Epoch 232/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0285 - accuracy: 0.9911 - val_loss: 0.2771 - val_accuracy: 0.9389
Epoch 233/700
281/281 [==============================] - 1s 4ms/step - loss: 0.0345 - accuracy: 0.9894 - val_loss: 0.2700 - val_accuracy: 0.9399

281/281 [==============================] - 1s 3ms/step - loss: 0.0230 - accuracy: 0.9922 - val_loss: 0.2689 - val_accuracy: 0.9439
Epoch 340/700
281/281 [==============================] - 2s 5ms/step - loss: 0.0217 - accuracy: 0.9933 - val_loss: 0.2645 - val_accuracy: 0.9369
Epoch 341/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0195 - accuracy: 0.9940 - val_loss: 0.2654 - val_accuracy: 0.9489
Epoch 342/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0181 - accuracy: 0.9935 - val_loss: 0.2692 - val_accuracy: 0.9399
Epoch 343/700
281/281 [==============================] - 2s 6ms/step - loss: 0.0168 - accuracy: 0.9944 - val_loss: 0.2661 - val_accuracy: 0.9439
Epoch 344/700
281/281 [==============================] - 1s 5ms/step - loss: 0.0190 - accuracy: 0.9935 - val_loss: 0.2768 - val_accuracy: 0.9399
Epoch 345/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0180 - accuracy: 0.9944 - val_loss: 0.2553 - val_accuracy: 0.9439

281/281 [==============================] - 2s 5ms/step - loss: 0.0123 - accuracy: 0.9961 - val_loss: 0.3137 - val_accuracy: 0.9309
Epoch 452/700
281/281 [==============================] - 1s 4ms/step - loss: 0.0131 - accuracy: 0.9956 - val_loss: 0.2995 - val_accuracy: 0.9339
Epoch 453/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0151 - accuracy: 0.9949 - val_loss: 0.2941 - val_accuracy: 0.9379
Epoch 454/700
281/281 [==============================] - 1s 5ms/step - loss: 0.0140 - accuracy: 0.9949 - val_loss: 0.2826 - val_accuracy: 0.9349
Epoch 455/700
281/281 [==============================] - 2s 6ms/step - loss: 0.0135 - accuracy: 0.9953 - val_loss: 0.2949 - val_accuracy: 0.9379
Epoch 456/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0151 - accuracy: 0.9944 - val_loss: 0.2995 - val_accuracy: 0.9399
Epoch 457/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0152 - accuracy: 0.9950 - val_loss: 0.2959 - val_accuracy: 0.9389

281/281 [==============================] - 1s 4ms/step - loss: 0.0103 - accuracy: 0.9961 - val_loss: 0.2799 - val_accuracy: 0.9429
Epoch 564/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0099 - accuracy: 0.9968 - val_loss: 0.2861 - val_accuracy: 0.9379
Epoch 565/700
281/281 [==============================] - 1s 4ms/step - loss: 0.0108 - accuracy: 0.9964 - val_loss: 0.2894 - val_accuracy: 0.9389
Epoch 566/700
281/281 [==============================] - 1s 4ms/step - loss: 0.0125 - accuracy: 0.9962 - val_loss: 0.2938 - val_accuracy: 0.9419
Epoch 567/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0079 - accuracy: 0.9976 - val_loss: 0.2915 - val_accuracy: 0.9379
Epoch 568/700
281/281 [==============================] - 1s 5ms/step - loss: 0.0142 - accuracy: 0.9949 - val_loss: 0.2959 - val_accuracy: 0.9389
Epoch 569/700
281/281 [==============================] - 1s 4ms/step - loss: 0.0132 - accuracy: 0.9949 - val_loss: 0.2912 - val_accuracy: 0.9429

281/281 [==============================] - 1s 5ms/step - loss: 0.0095 - accuracy: 0.9964 - val_loss: 0.3126 - val_accuracy: 0.9389
Epoch 676/700
281/281 [==============================] - 1s 4ms/step - loss: 0.0090 - accuracy: 0.9967 - val_loss: 0.3279 - val_accuracy: 0.9399
Epoch 677/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0099 - accuracy: 0.9966 - val_loss: 0.3323 - val_accuracy: 0.9419
Epoch 678/700
281/281 [==============================] - 2s 5ms/step - loss: 0.0090 - accuracy: 0.9970 - val_loss: 0.2966 - val_accuracy: 0.9439
Epoch 679/700
281/281 [==============================] - 2s 6ms/step - loss: 0.0081 - accuracy: 0.9971 - val_loss: 0.2981 - val_accuracy: 0.9409
Epoch 680/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0099 - accuracy: 0.9961 - val_loss: 0.2988 - val_accuracy: 0.9449
Epoch 681/700
281/281 [==============================] - 1s 3ms/step - loss: 0.0117 - accuracy: 0.9958 - val_loss: 0.3021 - val_accuracy: 0.9429

In [21]:
y_pred_model=model.predict(X_test)

32/32 [==============================] - 0s 1ms/step


In [22]:
y_pred_model

array([[3.7847799e-06, 9.4990904e-11, 3.0864378e-12, ..., 1.3639805e-12,
        1.2425652e-12, 9.9973112e-01],
       [1.3707747e-23, 1.9454211e-36, 0.0000000e+00, ..., 1.1632524e-12,
        1.1207025e-24, 4.6789218e-23],
       [9.9999797e-01, 8.3985796e-12, 1.4044859e-07, ..., 1.6685433e-12,
        1.9407510e-10, 1.9594643e-06],
       ...,
       [1.6078428e-05, 1.9213297e-15, 9.9556864e-15, ..., 2.2733565e-07,
        2.2274538e-07, 1.1987679e-07],
       [6.7495766e-06, 7.2020346e-01, 1.1590822e-06, ..., 4.5047319e-10,
        2.3668359e-11, 1.7808790e-06],
       [1.0000000e+00, 3.5691399e-28, 1.4201868e-22, ..., 8.2178512e-21,
        1.0663331e-12, 9.5931219e-17]], dtype=float32)

In [23]:
y_pred_model_new=[np.argmax(i) for i in y_pred_model]

df_result_y=pd.DataFrame({'yo':y_pred_model_new})
y_test.reset_index(inplace = True, drop = True)
y_test1=pd.DataFrame(y_test)
c=0
n=len(y_test1.iloc[:,0])
for i in range(0,len(y_test1.iloc[:,0])):
  one=y_test1.iloc[i,0]
  two=df_result_y.iloc[i,0]
  if(one==two):
    c=c+1
acc=c/n
print(acc)

0.9159159159159159


In [24]:
y_pred_model_2=model_2.predict(X_test)
y_pred_model_new_2=[np.argmax(i) for i in y_pred_model_2]

df_result_y_2=pd.DataFrame({'yo':y_pred_model_new_2})
y_test.reset_index(inplace = True, drop = True)
y_test1=pd.DataFrame(y_test)
c=0
n=len(y_test1.iloc[:,0])
for i in range(0,len(y_test1.iloc[:,0])):
  one=y_test1.iloc[i,0]
  two=df_result_y_2.iloc[i,0]
  if(one==two):
    c=c+1
acc=c/n
print(acc)

32/32 [==============================] - 0s 1ms/step
0.9359359359359359


In [25]:
y_pred_model_3=model_3.predict(X_test)
y_pred_model_new_3=[np.argmax(i) for i in y_pred_model_3]

df_result_y_3=pd.DataFrame({'yo':y_pred_model_new_3})
y_test.reset_index(inplace = True, drop = True)
y_test1=pd.DataFrame(y_test)
c=0
n=len(y_test1.iloc[:,0])
for i in range(0,len(y_test1.iloc[:,0])):
  one=y_test1.iloc[i,0]
  two=df_result_y_3.iloc[i,0]
  if(one==two):
    c=c+1
acc=c/n
print(acc)

32/32 [==============================] - 0s 1ms/step
0.9409409409409409


In [30]:

import lightgbm as lgb
d_train=lgb.Dataset(X_train, label=y_train)
#setting up the parameters
params={}
params['learning_rate']=0.3
params['boosting_type']='gbdt' #GradientBoostingDecisionTree
params['objective']='multiclass' #Multi-class target feature
params['metric']='multi_logloss' #metric for multi-class
params['max_depth']=10
params['num_class']=10 #no.of unique values in the target class not inclusive of the end value
#training the model
model_4=lgb.train(params,d_train,100)  #training the model on 100 epocs
y_pred_model_4=model_4.predict(X_test)




[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Accuracy may be bad since you didn't explicitly set num_leaves OR 2^max_depth > num_leaves. (num_leaves=31).
[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.002675 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14329
[LightGBM] [Info] Number of data points in the train set: 8991, number of used features: 57
[LightGBM] [Info] Start training from score -2.301585
[LightGBM] [Info] Start training from score -2.303809
[LightGBM] [Info] Start training from score -2.303809
[LightGBM] [Info] Start training from score -2.302696
[LightGBM] [Info] Start training from score -2.303809
[LightGBM] [Info] Start training from score -2.301585
[LightGBM] [Info] Start training from score -2.301585
[LightGBM] [Info] Start training from score -2.301585
[LightGBM] [Info

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf


In [31]:
y_pred_model_new_4=[np.argmax(i) for i in y_pred_model_4]
df_result_y_4=pd.DataFrame({'yo':y_pred_model_new_4})
y_test.reset_index(inplace = True, drop = True)
y_test1=pd.DataFrame(y_test)
c=0
n=len(y_test1.iloc[:,0])
for i in range(0,len(y_test1.iloc[:,0])):
  one=y_test1.iloc[i,0]
  two=df_result_y_4.iloc[i,0]
  if(one==two):
    c=c+1
acc=c/n
print(acc)

0.9329329329329329


In [28]:
y_pred_model_4

array([[7.92631602e-08, 1.51751394e-08, 8.92035124e-08, ...,
        2.03059878e-09, 1.20709030e-06, 9.99143393e-01],
       [4.52940397e-08, 1.13123851e-09, 2.80855446e-10, ...,
        1.31788182e-06, 2.41501835e-07, 2.61575957e-09],
       [9.99959462e-01, 3.59824966e-10, 3.93404932e-05, ...,
        1.21462711e-10, 7.32845057e-07, 4.38008827e-07],
       ...,
       [1.38077896e-04, 2.18082175e-08, 2.22833403e-06, ...,
        1.49710789e-06, 1.92290631e-04, 3.75346283e-05],
       [1.04157316e-02, 3.31979898e-01, 2.12520648e-04, ...,
        2.27997164e-03, 7.33867144e-05, 4.99840356e-05],
       [9.99999817e-01, 2.38966578e-12, 5.81096379e-09, ...,
        1.98955256e-11, 1.73114310e-07, 3.07461221e-09]])

In [32]:
from sklearn.ensemble import ExtraTreesClassifier, BaggingClassifier, RandomForestClassifier

In [33]:
et=ExtraTreesClassifier(random_state=11)

# Bagging Classifier
bgc = BaggingClassifier(et, random_state=11, verbose=0)

# Train 
y_train=y_train.astype('int')
model_5=bgc.fit(X_train, y_train)


In [34]:
y_pred_model_5=bgc.predict(X_test)

In [35]:
df_result_y_5=pd.DataFrame({'yo':y_pred_model_5})
y_test.reset_index(inplace = True, drop = True)
y_test1=pd.DataFrame(y_test)
c=0
n=len(y_test1.iloc[:,0])
for i in range(0,len(y_test1.iloc[:,0])):
  one=y_test1.iloc[i,0]
  two=df_result_y_5.iloc[i,0]
  if(one==two):
    c=c+1
acc=c/n
print(acc)

0.8938938938938938


In [158]:
model_json = model.to_json()
with open("model.json", "w") as json_file:
    json_file.write(model_json)
model.save_weights("model.h5")

In [159]:
model2_json = model_2.to_json()
with open("model2.json", "w") as json_file:
    json_file.write(model2_json)
model_2.save_weights("model2.h5")

In [160]:
model3_json = model_3.to_json()
with open("model3.json", "w") as json_file:
    json_file.write(model3_json)
model_3.save_weights("model3.h5")

In [161]:
import pickle
pickle.dump(model_4, open("model4.pkl", "wb"))

In [169]:
import pickle
pickle.dump(model_5, open("model5.pkl", "wb"))

In [86]:
# y_test.reset_index(inplace = True, drop = True)
# y_test1=pd.DataFrame(y_test)
# c=0
# n=len(y_test1.iloc[:,0])
# for i in range(0,len(y_test1.iloc[:,0])):
#   one=y_test1.iloc[i,0]
#   two=df_result_y_6.iloc[i,0]
#   if(one==two):
#     c=c+1
# acc=c/n
# print(acc)
#This is the validation accuracy on 10 percent data after ensembling the five models

0.9429429429429429
